# Using the weather-data package

The package supports downloading weather data from two sources:
* data in grib format from ECMWF MARS archive, 
* data in json format from [OpenWeatherMaps](https://openweathermap.org/) API.

Once downloaded, the data can be transformed and stored in other formats (csv).

## 1. Installation

Instructions can be found on the project's [wiki](https://github.com/JozefStefanInstitute/weather-data/wiki).

## 2. Download

Weather data can be downloaded from two different sources using the **WeatherApi** class.

The source is selected when creating the instance of **WeatherApi** (see example below) via 'source' arguments as:
* source='ecmfw' to download data from ECMWF or
* source='owm' to download data from Open Weather Map.

## 2.1. API key credentials

#### ECMWF

There are several possible ways to pass your ECMWF API key credentials.
1. You can store the credentials in a file - ***$HOME/.ecmwfapirc*** containing:

````json
{
  "url": "https://api.ecmwf.int/v1",
  "email": "<api_email>",
  "key": "<api_key>"
}
```
2. Or you can export the following environment variables:
    * export ECMWF_API_URL="https://api.ecmwf.int/v1"
    * export ECMWF_API_KEY="<api_key>"
    * export ECMWF_API_EMAIL="<api_email>"
3. Or you can pass the credentials directly as arguments when creating WeatherApi instance:

````python
wa = WeatherApi(source='ecmwf', key='<api_key>', email='<api_email>')
```

#### OWM

You can pass your OWM API key credentials as arguments when creating WeatherApi instance:

````python
wa = WeatherApi(source='owm', key='<api_key>')
```

## 2.2. Downloading the data

In [ ]:
import datetime
from weather.weather import WeatherApi

#### ECMWF

There are multiple arguments that can be specified when downloading the ECMWF data.

In [ ]:
wa = WeatherApi(source='ecmwf')

# Download weather forecast for July 2019 for Slovenia region
wa.get(from_date=datetime.date(2019, 7, 1), to_date=datetime.date(2019, 7, 31), 
       target='ecmwf-slovenia-july2019.grib', area=[46.53, 13.23, 45.25, 16.36])

#### Open Weather Map

The free version of OWM Api only provides current 5 day forecast for any location or city which includes weather data every 3 hours. The data is returned in **JSON** format.

One of following arguments needs to be specified on query:
* 'city_name' - city name and country code divided by comma, use ISO 3166 country codes
* 'city_id' - list of city ID city.list.json.gz can be downloaded from [OWM page](http://bulk.openweathermap.org/sample/)
* 'latlon' - lat, lon coordinates of the location of your interest 

More information about these fields can be found [here](https://openweathermap.org/forecast5).

In [ ]:
wa = WeatherApi(source='owm', api_key='2505b0ea15819fa575b8959799dd649f')

# By city name
wa.get(city_name='Ljubljana,si', target='owm-forecast-ljubljana.json') 

# By city id
wa.get(city_id=3196359, target='owm-forecast-ljubljana.json')

# By coordinates
wa.get(latlon=(46.0511, 14.5051), target='owm-forecast-ljubljana.json')

## 3. Query and Transform

Once the data is downloaded we can query it or transform it to ***csv*** format which can be then further imported to QMiner database.

In [ ]:
from weather.weather import WeatherExtractor

#### ECMWF

If **eccodes** is installed then it is used as a default GRIB file decoder, otherwise **pygrib** is used if available.

In [ ]:
we = WeatherExtractor()
we.load(['ecmwf-slovenia-july2019.grib'])

Get forecast made on 1.7.2019 for 2.7.2019, 3.7.2019 and 4.7.2019, for all grid points:


In [ ]:
weather_data = we.get_forecast(base_date=date(2019, 7, 1), from_date=date(
    2019, 7, 2), to_date=date(2019, 7, 4), aggtime='hour', aggloc='grid')
print(weather_data[:5])

We can also query the weather data for a specific point:

In [ ]:
points = [{'lat': 45.01, 'lon': 13.0}]
weather_data = we.get_actual(from_date=date(
    2019, 7, 2), to_date=date(2019, 7, 4), aggtime='hour', aggloc='points', interp_points=points)
print weather_data[:5]

In order to use weather data with QMiner we first have to export it as a ***.csv*** file which can later be imported in QMiner database.

In [ ]:
we.export('ecmwf-slovenia-july2019.tsv')

#### OWM

The same queries as with ECMWF can be used also in the OWN case.

**Note**: Currently multiple forecast for only **one location** are supported - we cannot load forecasts from Ljubljana, SI and Rome, IT simultaneously. 

In [ ]:
we = WeatherExtractor()
we.load(['owm-forecast-ljubljana.json'])